In [77]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)  # 修改这里，将x改为out
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

def process_data(data_dir):
    """
    原始数据处理
    param:
        data_dir: str, 数据集文件夹路径
    return:
        dataloader: DataLoader, 数据加载器
        class_names: list, 类别名称列表
    """
    batch_size=20
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    X = DataLoader(dataset, batch_size=20, shuffle=False)

    labels = [label for _, label in dataset] # 一个一个取图片滴label
    return X, labels





def predict(X):
    """
    模型预测
    param:
        model: nn.Module, 训练好的模型
        dataloader: DataLoader, 数据加载器
    return:
        y_predict: np.ndarray, 数据预测标签
    """

    device='cpu'
    all_predictions = []
    with torch.no_grad():  # 禁用梯度计算，节省内存和计算资源
        for data, target in X:
            # 将数据传递给模型进行推理
            output = model(data)

            # 假设模型输出是 logits，我们使用 softmax 获取概率
            probabilities = torch.softmax(output, dim=1)

            # 获取预测结果（最大概率的类别索引）
            predictions = torch.argmax(probabilities, dim=1)
            print( predictions )

            
            # 将预测结果转换为 NumPy 数组并保存
            all_predictions.extend(predictions.cpu().numpy())
            
    y_predict = all_predictions
    return y_predict

data_dir = './dataset/val'  # 替换为实际数据集路径
# 加载模型和权重
num_classes = 376  # 替换为您的分类数
model = ResNet18(num_classes=num_classes)
model.load_state_dict(torch.load('./ResNet.pt', map_location = torch.device('cpu') ))
model.eval()  # 设置模型为评估模式
dataloader, data_class_names = process_data(data_dir) 

In [78]:
# 群友写的类std后端
if __name__ == "__main__":
    X , labels = process_data(data_dir)
    y_predict = predict(X)
    # print( "labels:\n",labels )
    # print( "y_predict\n",y_predict )
    print(1111111111111)
    correct = 0
    total = len( labels )
    for true_label , pred_label in zip( labels , y_predict ):
        # print( f"测试中：{true_label}-----{pre_label}" )
        if true_label == pred_label:
            correct += 1
    accuracy = correct / total
    print(f"Accuarcy:{accuracy:.2%}")
        

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3])
tensor([3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7])
tensor([7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9])
tensor([ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10,
        10, 10])



KeyboardInterrupt



In [44]:
def process_data(data_dir):
    """
    原始数据处理
    param:
        data_dir: str, 数据集文件夹路径
    return:
        X: 经过预处理和特征选择后的特征数据
        labels: np.ndarray, 标签数据
    """

    import cv2
    import os
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    im_size = 150
    batch_size = 20

    # 将每个像素归一化
    images = ImageDataGenerator(rescale = 1/255)

    # 载入图片，批处理大小，shuffle，size
    img = images.flow_from_directory(directory=data_dir,
                                                 batch_size=batch_size,
                                                 shuffle=False,
                                                 target_size=(im_size, im_size),
                                                 class_mode=None)# 载入图片，批处理大小，shuffle，size
    X = img
    labels = img.classes

    return X, labels


X , labels = process_data('./dataset/val')

total_samples = X.samples
print(total_samples)
print(f"labels: {len(labels)}")

print( type(labels) )



Found 3988 images belonging to 376 classes.
3988
labels: 3988
<class 'numpy.ndarray'>
